In [1]:
%load_ext autoreload

%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from spikeA.Session import *
from spikeA.Spike_train_loader import Spike_train_loader
from spikeA.Cell_group import Cell_group
from spikeA.Animal_pose import Animal_pose
from spikeA.Session import Session

In [ ]:
#name="pk1330-08112021-0101"
#name="pk1330-10112021-0101"
#name="pk1330-16112021-0101"

#name="pk1330-12112021-0101"
#name="pk1331-23112021-0101"
#name="pk1331-23112021-0101_B"

#name="pk1331-24112021-0101"
#name="pk1347-24112021-0101"

#name="pk1331-29112021-0101"
#name="pk1347-01122021-0101"

#name="pk1658-06122021-0102"

#name="pk1331-29112021-0101" # HD cells
#name="pk1658-10122021-0101"

#name="pk1613-12012022-0101"
#name="pk1613-13012022-0101"

#name="pk1615-14012022-0101"

#name="pk1615-15012022-0101"
#name="pk1615-16012022-0101"
#name="pk1615-17012022-0101"
#name="pk1615-18012022-0101_A"

#name="pk1615-18012022-0103"
#name="pk1615-19012022-0105"
#name="pk1615-19012022-0102"

#name="pk1615-20012022-0101_B"
#name="pk3865-20012022-0101_B"

# name="pk3865-21012022-0101"
name="pk1615-21012022-0109"

In [ ]:
# mouse name = .... , plot on summary file

directory, session = name.split("-",1)
path="/adata/electro/"+directory+"/"+name+"/"

fname = name.split("_")[0]

print("Path:",path)
print("Name:",name,",",fname)

import os
print("directory:", "OK" if os.path.isdir(path) else "invalid")

In [4]:
path="/home/pascal/Downloads/pk1615-21012022-0109/"
name="pk1615-21012022-0109"
fname=name

Check the files

## Get a Session object and load spikes

In [5]:
ses = Kilosort_session(name=fname,path=path)
ses.load_parameters_from_files()
print(ses)

ValueError: The file /home/pascal/Downloads/pk1615-21012022-0109/pk1615-21012022_01.dat is missing.

In [ ]:
ses.trial_intervals.inter

In [ ]:
ses.desen

In [ ]:
for en,iv,tn in zip(ses.desen, ses.trial_intervals.inter, ses.trial_names):
    print("trial:",tn)
    print("environment:",en)
    print("duration:",iv[1]-iv[0])

In [ ]:
ses.desel

In [ ]:
stl = Spike_train_loader()
stl.load_spike_train_kilosort(ses)
print("stl.clu_ids",stl.clu_ids)
cg = Cell_group(stl)

In [ ]:
print("Found ",len(cg.neuron_list), "good clusters (neurons)")

for i,(clu_id, spike_time) in enumerate(zip(stl.clu_ids, stl.spike_times)):
    print("#",i,", id =",clu_id, ", spikes:",len(spike_time))

## Load the waveforms

In [ ]:
ses.load_waveforms()

Plot the waveforms for one arbitrary cluster (neuron) on the channels of its highest peak-to-peak amplitudes

In [ ]:
#cluster = 16
cluster = stl.clu_ids[0]
channels = ses.get_channels_from_cluster(cluster, 12)
print("The most significant channels for cluster",cluster,":")
print(channels)

In [ ]:
# plot on selected channels

plt.figure(figsize=(10,15))

for i,channel in enumerate(channels):
    
    channel_mapped, waveform = ses.get_waveform(cluster, channel)
    
    plt.axhline( y = -i*.5 , color="grey" , lw=.5 )
    label = "channel: "+str(channel)+" ("+str(channel_mapped)+")"
    plt.plot( waveform -i*.5 , label=label , color="blue")
    plt.text( s= label , x=0, y=-i*.5 +.1 )
plt.xlabel("time sample")
#plt.legend()
ax = plt.gca()
ax.set(yticklabels=[])  
plt.title("Waveform of Cluster "+str(cluster))
plt.show()

In [ ]:
# plot all channels

channels_all = range(ses.wv_channels)

plt.figure(figsize=(10,15))

for i,channel in enumerate(channels_all):
    channel_mapped, waveform = ses.get_waveform(cluster, channel)
    
    plot_style = [2.,'bold'] if channel in channels else [.9, 'normal']
    
    plt.axhline( y = -i*.5 , color="grey" , lw=.5 )
    label = "channel: "+str(channel)+" ("+str(channel_mapped)+")"
    plt.plot( waveform -i*.5 , label=label , color="blue" , lw=plot_style[0])
    plt.text( s= label , x=0, y=-i*.5 +.1 , weight=plot_style[1])
plt.xlabel("time sample")
#plt.legend()
ax = plt.gca()
ax.set(yticklabels=[])  # remove the tick labels
ax.set(ylabel=None)  # remove the y-axis label
ax.tick_params(left=False)  # remove the ticks

plt.title("Waveform of Cluster "+str(cluster))
plt.show()

## Channel position (alignment)

In [ ]:
ses.init_shanks()

In [ ]:
fig, ax = plt.subplots(figsize=(9,12))

for channel, position in zip(range(ses.wv_channels), ses.channel_positions):
    print("channel:",channel," (",ses.channel_map[channel],"), position:",position)
        
    circle = plt.Circle((position[0], position[1]), 0.2, color="black")
    ax.add_patch(circle)
    
    plt.text(x=position[0]+1, y=position[1], s="channel:"+str(channel)+" ("+str(ses.channel_map[channel])+")")
    

ax.axis('off')
ax.axis('scaled') #this line fits your images to screen 
#ax.autoscale(enable=True) 
plt.title("Channel positions")
plt.tight_layout()
plt.show()

draw this for the specific cluster showing the significant channels

In [ ]:
shanks_all = np.unique(ses.channel_positions[:,0])
print("Shanks:",shanks_all)
shanks_arr = np.zeros(len(shanks_all))

In [ ]:
active_shanks = np.unique(ses.channel_positions[channels][:,0])
print("active_shanks",active_shanks)

In [ ]:
shanks_arr[[list(shanks_all).index(shank) for shank in active_shanks]]=1
shanks_arr

In [ ]:
fig, ax = plt.subplots(figsize=(9,12))

for channel, position in zip(range(ses.wv_channels), ses.channel_positions):        
    color = 'blue' if channel in channels else 'gray'
    
    circle = plt.Circle((position[0], position[1]), 0.2, color=color)
    ax.add_patch(circle)
    
    shank_id = position[0] # = x coordinate of the position
    if channel in channels:
        shanks_arr[np.where(shanks_all==shank_id)[0][0]]=1 # mark shank as active for this cluster
    
    plt.text(x=position[0]+1, y=position[1], s="channel:"+str(channel)+" ("+str(ses.channel_map[channel])+")")
    

ax.axis('off')
ax.axis('scaled') #this line fits your images to screen 
#ax.autoscale(enable=True) 
plt.title("Channel positions for cluster "+str(cluster))
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(9,3))

for i, (shank, active) in enumerate(zip(shanks_all, shanks_arr)):
    print("shank",str(int(shank)),"is","active" if active else "nope")
    
    color = 'blue' if active else 'gray'
    
    circle = plt.Circle((i,0), 0.2, color=color)
    ax.add_patch(circle)
    
    # plt.text(x=i, y=-1, s="shank:"+str(int(shank)))

ax.axis('off')
ax.axis('scaled') #this line fits your images to screen 
#ax.autoscale(enable=True) 
plt.title("Active Shanks for cluster "+str(cluster))
plt.tight_layout()
plt.show()

## Create a pose file 

In [ ]:
ap = Animal_pose(ses)
if not os.path.exists(f"{ses.fileBase}.pose.npy"):
    print("load from positrack")
    ap.pose_from_positrack_files(ses=ses, interpolation_frequency_hz = 50, extension="positrack")
    ap.save_pose_to_file() # save the file for next time
else:
    print("load from saved pose file")
    ap.load_pose_from_file()

In [ ]:
ap.pose_from_positrack_files(ses=ses, interpolation_frequency_hz = 50, extension="positrack")

In [ ]:
np.nanmax(ap.pose[:,6])

In [ ]:
ap.ttl_ups[0].shape

In [ ]:
ap.ttl_ups

Plot the path of the animal to see if it was loaded correctly.

In [ ]:
plt.plot(ap.pose[:,1],ap.pose[:,2])
plt.xlabel("x")
plt.ylabel("y")
plt.gca().set_aspect('equal', adjustable='box')

Plot the occupancy map to see where the animal spent more or less time.

(The image seems mirrored since the x/y-axis for imshow is different compared to the plot function.)

In [ ]:
ap.occupancy_map_2d()
plt.imshow(ap.occupancy_map)
plt.show()

Plot the head-direction of the animal and its position a short segment.

In [ ]:
s=3000
e=3800
plt.plot(ap.pose[s:e,4])
plt.ylabel("Head-direction")
plt.show()
plt.plot(ap.pose[s:e,1],ap.pose[s:e,2])

Now we need to set the SpatialProperties object of each Neuron objects of the CellGroup.
Then we can get information score for each neuron from a firing rate map without smoothing.
We recalculate the maps with smoothing for display purposes.

In [ ]:
ap.head_direction_occupancy_histogram()
plt.plot(ap.hd_occupancy_bins[:-1],ap.hd_occupancy_histogram)
plt.ylim(0,np.max(ap.hd_occupancy_histogram)+2)
plt.ylabel("Time (sec)")
plt.xlabel("Head direction (deg)")

In [ ]:
print("raio invalid HD =" , sum(np.isnan(ap.pose[:,4])) / len(ap.pose[:,4]))

In [ ]:
## demo execution for one neuron

# polar plot
n = cg.neuron_list[1]
n.set_spatial_properties(ap)
n.spatial_properties.firing_rate_head_direction_histogram(smoothing=False)

angles = n.spatial_properties.mid_point_from_edges(n.spatial_properties.firing_rate_head_direction_histo_edges)
hd_firing = n.spatial_properties.firing_rate_head_direction_histo

hd_mean_direction_rad, hd_mean_direction_deg, hd_mean_vector_length, hd_peak_angle_rad, hd_peak_rate = n.spatial_properties.head_direction_score()

#print("angles:",len(angles),angles)
#print("hd_firing:",len(data),data)

print("hd_mean_direction_rad, hd_mean_direction_deg, hd_mean_vector_length",hd_mean_direction_rad, hd_mean_direction_deg, hd_mean_vector_length)

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
#ax.plot(angles, data)
ax.plot(np.append(angles,angles[0]), np.append(hd_firing,hd_firing[0]))
ax.grid(True)
#ax.axvline(hd_mean_direction_rad, hd_mean_vector_length, color="black")
ax.set_title("Head direction")
plt.show()

In [ ]:

for i,(tn,iv,en) in enumerate(zip(ses.trial_names, ses.trial_intervals.inter, ses.desen)):
    
    print("Trial",i,"("+tn+")")
    
    print("environment:",en)
    
    if en=="rb":
        continue
    
    iv=np.array([iv])
    print("Interval:",iv)

    # reset and set intervals
    n.spike_train.unset_intervals()
    ap.unset_intervals()
    n.spike_train.set_intervals(iv)
    ap.set_intervals(iv)
    n.set_spatial_properties(ap)
    
    # occupancy map
    ap.occupancy_map_2d()
    plt.imshow(ap.occupancy_map)
    plt.title("occupancy map")
    #plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().axis('equal')
    plt.show()
    print("occupancy map",type(ap.occupancy_map),ap.occupancy_map.shape)
    
    
    #xy_range = np.array([[0,0],[70,70]]) # [[xmin,ymin],[xmax,ymax]]
    xy_range = ap.invalid_outside_spatial_area(shape = "square", length=70)
    print("xy_range",xy_range)
    ap.occupancy_map_2d(xy_range=xy_range)
    plt.imshow(ap.occupancy_map)
    plt.title("occupancy map")
    #plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().axis('equal')
    plt.show()
    print("occupancy map",type(ap.occupancy_map),ap.occupancy_map.shape)
    
    # spatial firing map 
    n.spatial_properties.firing_rate_map_2d(cm_per_bin =2, smoothing_sigma_cm = 2, smoothing=True)
    plt.imshow(n.spatial_properties.firing_rate_map,cmap='jet')
    plt.title("clu:{}, mRate:{:.3} Hz".format(n.name,n.spike_train.mean_firing_rate()))
    plt.show()
   
    print("")


In [ ]:
ses.n_trials

In [ ]:
trials_environments = [ en for en in ses.desen if not en=="rb" ]

In [ ]:
trials_environments

In [ ]:
n.spike_train.st

In [ ]:

fig, axs = plt.subplots(len(cg.neuron_list), len(trials_environments), figsize=(25, 5*len(cg.neuron_list)))

for i, n in enumerate(cg.neuron_list):
    
    n.set_spatial_properties(ap)
    
    j=0
    
    for t,(tn,iv,en) in enumerate(zip(ses.trial_names, ses.trial_intervals.inter, ses.desen)):
        
        n.set_spatial_properties(ap)

        #print("Trial",t,"("+tn+")")

        #print("environment:",en)
        
        if en=="rb":
            continue


        iv=np.array([iv])
        #print("Interval:",iv)

        # reset and set intervals
        #n.spike_train.unset_intervals()
        #ap.unset_intervals()
        n.spike_train.set_intervals(iv)
        ap.set_intervals(iv)

        # spatial firing map 
        n.spatial_properties.firing_rate_map_2d(cm_per_bin =2, smoothing_sigma_cm = 2, smoothing=True)
        
        axs[i,j].imshow(n.spatial_properties.firing_rate_map,cmap='jet')
        gs=n.spatial_properties.grid_score()
        
        fw='bold' if gs>.4 else 'normal'
            
        axs[i,j].set_title("clu:{}, mRate:{:.3} Hz, grid score:{:.3}".format(n.name,n.spike_train.mean_firing_rate(),gs) + "\n" + "Trial "+str(t+1)+" ("+tn+")"+"\n"+"environment: "+en, fontweight=fw)
        j+=1
        

        #print("")
        
    #testing
    #if i>2:
    #    break

plt.tight_layout()
fig.subplots_adjust(wspace=0.33, hspace=.5)
plt.savefig('/home/pascal/Documents/science_docs/analysis_results/'+name+'_trials.pdf')
plt.show()


In [ ]:
# map correlation

print("map correlation")

for i, n in enumerate(cg.neuron_list):
    
    print(i,n.name)
    
    n.set_spatial_properties(ap)    
    maps=[] # maps to compare with
    
    for t,(tn,iv,en) in enumerate(zip(ses.trial_names, ses.trial_intervals.inter, ses.desen)):

        print("Trial",t,"("+tn+")")

        print("environment:",en)
        
        if en=="rb":
            continue

        iv=np.array([iv])
        print("Interval:",iv)
        
        # reset and set intervals
        #n.spike_train.unset_intervals()
        #ap.unset_intervals()
        n.spike_train.set_intervals(iv)
        ap.set_intervals(iv)

        # spatial firing map 
        # xy_range=[[xmin,ymin],[xmax,ymax]]
        ## if t==2:
        ##     xy_range=np.array([[7,1],[44,36]])
        ## else:
        ##    xy_range=None
        ## n.spatial_properties.firing_rate_map_2d(cm_per_bin =2, smoothing_sigma_cm = 2, smoothing=True, xy_range=xy_range)
        
        
        n.spatial_properties.ap.invalid_outside_spatial_area(shape = "square", length=70)
        n.spatial_properties.firing_rate_map_2d(cm_per_bin =2, smoothing_sigma_cm = 2, smoothing=True)
        
        
        firing_rate_map = n.spatial_properties.firing_rate_map
        
        if t==0:
            map1 = firing_rate_map
        else:
            maps.append(firing_rate_map)
        

        #print("")
        
    #testing
    #if i>2:
    #    break
    
    for map2 in maps:
        print("map1",map1.shape)
        plt.imshow(map1)
        plt.title("map1")
        plt.show()
        print("map2",map2.shape)
        plt.imshow(map2)
        plt.title("map2")
        plt.show()
        p = n.spatial_properties.map_crosscorrelation(map1=map1, map2=map2)
        print("p=",p)

Loop for our neurons and do the analysis we need to know if we have spatially selective neurons or head-direction cells

In [ ]:
ses.trial_intervals.inter

In [ ]:
hd_firing_list=[] # neurons tuning curve
hd_ifr_list=[] # ifr list
spike_trains = [] # spike trains


# from matplotlib.patches import Rectangle

cols = 8

fig, axs = plt.subplots(len(cg.neuron_list), cols, figsize=(25, 5*len(cg.neuron_list)))
   
for i, n in enumerate(cg.neuron_list):
    
    # temporal auto-correlation
      
    n.spike_train.spike_time_autocorrelation(bin_size_sec=0.0005,min_sec=-0.05, max_sec=0.05)
    timestamp = n.spike_train.mid_point_from_edges(n.spike_train.st_autocorrelation_histogram[1])
    axs[i,0].plot(timestamp, n.spike_train.st_autocorrelation_histogram[0])
    
    n.spike_train.spike_time_autocorrelation(bin_size_sec=0.0025,min_sec=-0.5, max_sec=0.5)
    timestamp = n.spike_train.mid_point_from_edges(n.spike_train.st_autocorrelation_histogram[1])
    axs[i,1].plot(timestamp, n.spike_train.st_autocorrelation_histogram[0])
    
    
    # clear intervals
    
    n.spike_train.unset_intervals()
    ap.unset_intervals()
    n.spike_train.set_intervals(ses.trial_intervals.inter)
    ap.set_intervals(ses.trial_intervals.inter)
    
    # spatial firing map 
    
    n.set_spatial_properties(ap)
    n.spatial_properties.firing_rate_map_2d(cm_per_bin =2, smoothing_sigma_cm = 2, smoothing=True)
    im = axs[i,2].imshow(n.spatial_properties.firing_rate_map,cmap='jet')
    axs[i,2].set_title("clu:{}, mRate:{:.3} Hz".format(n.name,n.spike_train.mean_firing_rate()))
    fig.colorbar(im,ax=axs[i,2])
    
    
    # Spatial autocorrelation Map & Grid Score
    gs = n.spatial_properties.grid_score()
    
    fw = "bold" if gs>.4 else "normal"
    
    axs[i,3].imshow(n.spatial_properties.spatial_autocorrelation_map)
    axs[i,3].set_title("spatial autocorrelation"+"\n"+"Grid Score: "+str(round(gs,2)), fontweight=fw)
    
    # Head direction
    
    n.spatial_properties.firing_rate_head_direction_histogram(smoothing=False)
    
    angles = n.spatial_properties.mid_point_from_edges(n.spatial_properties.firing_rate_head_direction_histo_edges)
    hd_firing = n.spatial_properties.firing_rate_head_direction_histo
    hd_mean_direction_rad, hd_mean_direction_deg, hd_mean_vector_length, hd_peak_angle_rad, hd_peak_rate = n.spatial_properties.head_direction_score()
    
    axs[i,4].plot(angles, hd_firing)
    axs[i,4].set_xlabel("Head direction")
    axs[i,4].set_ylabel("Firing rate (Hz)")
    axs[i,4].set_ylim(0,np.max(n.spatial_properties.firing_rate_head_direction_histo)*1.2)
    axs[i,4].axvline(x=hd_mean_direction_rad, color="black")
    #axs[i,4].text(s=str(round(hd_mean_direction_rad*180./np.pi))+" ° (vector length = "+str(hd_mean_vector_length)+")",x=hd_mean+.1,y=1)
    
    fw = "bold" if hd_mean_vector_length>.4 else "normal"
    axs[i,5].axis('off')
    ax2 = fig.add_subplot(len(cg.neuron_list), cols, cols*i+6, polar=True)
    #ax2.plot(angles, hd_firing)
    ax2.plot(np.append(angles,angles[0]), np.append(hd_firing,hd_firing[0]))
    ax2.plot([hd_mean_direction_rad]*2, [0,max(hd_firing)] , color="blue", alpha=hd_mean_vector_length, lw=2.5  )
    ax2.set_title("Head direction" + "\n" + "mean direction: " + str(round( (hd_mean_direction_deg + 360) % 360  )) +"°" + "\n" + "mean vector length: " + str(np.round(hd_mean_vector_length,2))  , fontweight=fw) 
    ax2.grid(True)
    
    
    # Wavefroms
    
    cluster = int(n.name) # current cluster name
    channels = ses.get_channels_from_cluster(cluster) # get channels with highest amplitude
    
    if (len(channels)):
    
        for j,channel in enumerate(channels):
            channel_mapped, waveform = ses.get_waveform(cluster, channel)
            axs[i,6].axhline( y = -j*.5 , color="grey" , lw=.5 )
            #label = "channel: "+str(channel)+" ("+str(channel_mapped)+")"
            label = "channel "+str(channel_mapped)
            axs[i,6].plot( waveform -j*.5 , label=label , color="blue")
            axs[i,6].text( s= label , x=0, y=-j*.5 +.1 )
        axs[i,6].set_xlabel("time sample")
        axs[i,6].set(yticklabels=[])  
        axs[i,6].set_title("Waveform of Cluster "+str(cluster))
        
    # shanks
    active_shanks, shanks_arr = ses.get_active_shanks(channels)
   
    for j, (shank, active) in enumerate(zip(shanks_all, shanks_arr)):
        color = 'blue' if active else 'gray'
        circle = plt.Circle((j,0), 0.2, color=color)
        axs[i,7].add_patch(circle)

    
    axs[i,7].axis('off')
    axs[i,7].set_title("Active Shanks for cluster "+str(cluster)+"\n"+', '.join([str(int(active_shank)) for active_shank in active_shanks]))
    
    axs[i,7].set_xlim([-1, len(shanks_all)])
    axs[i,7].set_ylim([-0.5, 0.5])
    
    ###########################
    # classify, find category: Grid cell, HD cell
    
    if (hd_mean_vector_length > 0.4):
        n.spike_train.instantaneous_firing_rate(bin_size_sec=2.e-2)
        print("n.name",n.name)
        print("n.spike_train.ifr",n.spike_train.ifr)
        print("n.spike_train.st",n.spike_train.st)
        
        hd_firing_list.append(hd_firing)
        hd_ifr_list.append(n.spike_train.ifr)
        spike_trains.append(n.spike_train.st)
        
        #rect = Rectangle((0.08, 1-(i+.5)/len(cg.neuron_list)), 0.85, 1/len(cg.neuron_list), facecolor='yellow', edgecolor='none', transform=fig.transFigure, zorder=-1)
        #fig.patches.append(rect)

    
    
#plt.tight_layout()
fig.subplots_adjust(wspace=0.33, hspace=.5)
plt.savefig('/home/pascal/Documents/science_docs/analysis_results/'+name+'.pdf')
plt.show()


In [ ]:
angles = np.array(angles)
print("angles.shape",angles.shape)

hd_firing_list = np.array(hd_firing_list)
print("hd_firing_list.shape",hd_firing_list.shape)

hd_ifr_list = np.array(hd_ifr_list)
print("hd_ifr_list.shape",hd_ifr_list.shape)

spike_trains = np.array(spike_trains)
print("spike_trains.shape",spike_trains.shape)
'''
np.save("angles.npy",angles)
np.save("hd_firing_list.npy",hd_firing_list)
np.save("hd_ifr_list.npy",hd_ifr_list)
np.save("ap-pose.npy",ap.pose)
np.save("spike_trains.npy",spike_trains)
#'''
plt.plot(hd_ifr_list[0,2])
ap.pose[:,0].shape
hd_ifr_list[0,2,:].shape
ap.pose[:,0]
hd_ifr_list[0,2,:]